## Feature Extraction, New and Improved

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
from pathlib import Path
from sklearn.model_selection import train_test_split
from pydub import AudioSegment
from scipy.fftpack import fft, dct

---

## Visualize audio data with librosa

In [ ]:
y, sr = librosa.load('../../../Source/Shuffle_test/1/1_pad.wav')
D = np.abs(librosa.stft(y))

# Using left-aligned frames
D_left = np.abs(librosa.stft(y,center=False))

# Using a shorter hop length
D_short = np.abs(librosa.stft(y, n_fft=n_fft,hop_length=64)) # defaults to win_length(or n_fft)/4.



#librosa.display.specshow(D_left);
#librosa.display.specshow(D_short);

# Wave plot
plt.figure(figsize=(8,6))
plt.subplot(221)
librosa.display.waveplot(y, sr=sample_rate)
plt.title('Raw data: amplitude')

# Power spectrogram with abs(stft(y))^2
plt.subplot(222)
librosa.display.specshow(D**2, sr=sr, y_axis='log')
plt.colorbar()
plt.title('Power spectrogram')

# Log power with power_to_db
plt.subplot(223)
librosa.display.specshow(librosa.power_to_db(D**2,ref=np.max),sr=sr, y_axis='log', x_axis='time')
plt.title('Log power spectrogram')
plt.colorbar(format='%+2.0f dB')

# Mel spectrogram 
plt.subplot(224)
librosa.display.specshow(librosa.power_to_db(D**2,ref=np.max),
                         y_axis='mel', fmax=8000, x_axis='time')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel spectrogram');
plt.tight_layout()



#y, sr = librosa.load('../../../workout.wav')
#D = np.abs(librosa.stft(y))**2
#S = librosa.feature.melspectrogram(S=D)
#
#
#plt.figure(figsize=(10, 4))
#librosa.display.specshow(librosa.power_to_db(S,ref=np.max),
#                         y_axis='mel', fmax=8000, x_axis='time')
#plt.colorbar(format='%+2.0f dB')
#plt.title('Mel spectrogram: ("Workout" by Chance the rapper)')
#plt.tight_layout()

---

## Pad samples with zeros to create equal length

In [93]:
path_col = []
pathlist = Path('../../../Source/Clean_train_clips/Re_augmented_pad').glob('**/*.wav')
#pathlist = Path('../../../Source/Clean_train_clips/Test_pad').glob('**/*.wav')
for path in pathlist:
    path_col.append(path)

In [39]:
# Test Data 
#X_reserve = pd.read_csv('../../../Source/Data/X_test_reserved.csv')


#path_col = []
#for i in range (len(X_reserve)):
#    path_col.append(Path(X_ddd.loc[i,'Path']))
#

In [94]:
#path_col
len(path_col)

912

In [38]:
length_list = []
for i in range (len(path_col)):
    samples, sample_rate = librosa.load(path_col[i])
    length_list.append(len(samples))
max_length = max(length_list)
print(max_length)

20772


In [59]:
def pad_signal(path, length):
    samples, sample_rate = librosa.load(path)
    #name = str(path)[:34] + "Test_pad/" + str(path)[34:]
    #name = path
    if len(samples) < length:
        y = librosa.util.pad_center(samples, length ,axis=0) 
    else:
        y = samples
    return librosa.output.write_wav(path=name, y=y, sr=sample_rate)

In [60]:
# Pad with silence and resave (saving over file)
#max_length = 20772
#for i in range(len(path_col)):
#    pad_signal(path_col[i], max_length)

In [63]:
# Check if it worked
for path in path_col:
    samples, sample_rate = librosa.load(path)
    print(librosa.core.get_duration(y=samples, sr=sample_rate))


0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163265306
0.9420408163

---

# Feature Extraction: exploration
## Tempo

Let's try it with a stretched and unstretched sample

In [ ]:
fast_clip = ('../../../Source/Clean_train_clips/Shuffle/1/1.wav')
slow_clip = ('../../../Source/Clean_train_clips/Augmented/Shuffle/1/1_stretch.wav') 

In [ ]:
def get_features_bpm(path): 
    samples, sample_rate = librosa.load(path)
    onset_env = librosa.onset.onset_strength(samples, sr=sample_rate) # Assumes static tempo, for dynamic: aggregate=None
    tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sample_rate)
    return tempo.item()


In [ ]:
get_features_bpm(slow_clip), get_features_bpm(fast_clip)

## Short-time Fourier transform (STFT)

From librosa documentation: <br/>
Returns a complex-valued matrix D such that
np.abs(D[f, t]) is the magnitude of frequency bin f at frame t

In [ ]:
n_fft = 2048 # default 2048


In [ ]:
def get_features_stft (path):
        y, sr = librosa.load(path)
        D = np.abs(librosa.stft(y))
        return D

## Mel-frequency cepstral coefficients 

In [97]:
n_mfcc = 20 

In [98]:
def get_features_mfcc(path):
    samples, sample_rate = librosa.load(path)
    mfccs = np.mean(librosa.feature.mfcc(y=samples, sr=sample_rate, n_mfcc=n_mfcc).T,axis=0)
    return mfccs

## Zero crossing rate
Use with padded samples of equal length

In [64]:
def get_features_ZCR(path):
    samples, sample_rate = librosa.load(path)
    return librosa.feature.zero_crossing_rate(samples, frame_length=250, hop_length=125)

In [68]:
test_path = ('../../../Source/Clean_train_clips/Test_pad/Shuffle/1/10.wav')

In [69]:
zcr_length = get_features_ZCR(test_path).shape[1]

In [70]:
zcr_length

167

---

## Feature Extraction: implementation
(Organized in a DataFrame)

In [99]:
def build_list(step, folder, length):
    i = 1
    step_list = []
    while i <= length :
        name = step + "/" + str(folder) + "/" +str(i) + ".wav"
        step_list.append(name)
        i += 1
    return step_list

def get_label(path):
    if path.parts[-3] == 'Shuffle':
        return 1
    else:
        return 0

---

### Load Training Data

In [100]:
shuffle_col, bc_col, path_col = [], [], []

In [101]:
pathlist = Path('../../../Source/Clean_train_clips/Test_pad/Shuffle').glob('**/*.wav')
for path in pathlist:
    shuffle_col.append(path)
    shuffle_col.sort()

In [102]:
pathlist = Path('../../../Source/Clean_train_clips/Test_pad/Ball_change').glob('**/*.wav')
for path in pathlist:
    bc_col.append(path)
    bc_col.sort()

In [ ]:
#untrans_path_col = []
#train_untransform = pd.read_csv('../../../Source/Data/X_train_preAugmented.csv')
#for i in range ( len(train_untransform)):
#    untrans_path_col.append(Path(train_untransform.loc[i, 'Path']))

In [103]:
path_col = shuffle_col + bc_col #+ untran's_path_col

In [104]:
len(path_col)

115

** Create DataFrame from file paths **

In [105]:
tap = pd.DataFrame({'Path':path_col})
tap.shape

(115, 1)

---

### OR load (reserved) Test Data

In [ ]:
#test_data = pd.read_csv('../../../Source/Data/X_test_reserved.csv')
#test_labels = pd.read_csv('../../../Source/Data/y_test_reserved.csv')

In [ ]:
# tap = test_data

---

**Add labels**

In [106]:
tap['Labels'] = [get_label(tap.loc[idx,'Path']) for idx in range(len(tap))]

In [107]:
tap.head()

,Path,Labels
0,../../../Source/Clean_train_clips/Test_pad/Shu...,1
1,../../../Source/Clean_train_clips/Test_pad/Shu...,1
2,../../../Source/Clean_train_clips/Test_pad/Shu...,1
3,../../../Source/Clean_train_clips/Test_pad/Shu...,1
4,../../../Source/Clean_train_clips/Test_pad/Shu...,1


**Add Features: MFCCs**

In [108]:
# TOO SLOW!
#for i in range (n_mfcc):
#    tap[str(i)] = [get_features_mfcc(tap.loc[idx, 'Path'])[i] for idx in range (len(tap))]

In [109]:
# Create an empty dataframe to fill with MFCC values
d = pd.DataFrame(np.zeros((len(tap), n_mfcc)))
tap = pd.concat([tap, d], axis=1)

In [110]:
# Add feature to DataFrame
for j in range (len(tap))  :  
    s = get_features_mfcc(tap.loc[j,'Path'])
    for i in range (n_mfcc):
        tap.iat[j,i+2] = s[i]

In [111]:
tap.head()

,Path,Labels,0,1,2,3,4,5,6,7,...,10,11,12,13,14,15,16,17,18,19
0,../../../Source/Clean_train_clips/Test_pad/Shu...,1,-335.976138,43.414944,-68.477188,21.008180,-4.408123,-2.970243,-19.639680,-0.455080,...,-12.835432,-8.566720,-0.702506,-5.645249,-9.004763,-0.480133,-4.021731,-9.929621,-2.359306,-1.650723
1,../../../Source/Clean_train_clips/Test_pad/Shu...,1,-351.007970,40.109784,-63.794729,23.560631,-2.753592,-0.453999,-15.169304,-3.271940,...,-20.223677,-6.160998,0.380872,-5.626927,-9.627007,-1.699224,-2.785877,-13.520097,-5.137728,-1.594331
2,../../../Source/Clean_train_clips/Test_pad/Shu...,1,-497.674053,19.656596,-27.546274,10.032751,-4.512792,0.937636,-7.593287,1.180707,...,-9.038659,-2.498340,3.109148,-5.935463,-2.240996,-2.418737,-3.276783,-6.091257,-0.218293,-3.540506
3,../../../Source/Clean_train_clips/Test_pad/Shu...,1,-367.690502,42.046789,-42.360295,15.060588,-14.119677,7.645445,-11.598609,-0.446276,...,-17.959095,-4.773915,-2.184383,-8.884320,-7.921833,-3.847076,-3.137474,-10.508543,-2.878794,-2.108435
4,../../../Source/Clean_train_clips/Test_pad/Shu...,1,-370.150079,49.300692,-44.129399,14.314299,-7.690344,3.293678,-19.717559,-2.202616,...,-15.074157,-1.624259,-0.397585,-7.367681,-4.400169,-4.383350,-5.219379,-8.893206,-1.945035,-0.631430


** Add Features: Tempo**

In [ ]:
tap['BPM'] = [get_features_bpm(tap.loc[idx, 'Path']) for idx in range (len(tap))]

** Add features: Zero-crossing rate**

In [81]:
# Create an empty dataframe to fill with ZCR values
zcr_length = zcr_length
d = pd.DataFrame(np.zeros((len(tap), zcr_length)))
tap = pd.concat([tap, d], axis=1)

In [82]:
for j in range (len(tap))  :  
    s = get_features_ZCR(tap.loc[j,'Path'])[0]
    for i in range (zcr_length):
        tap.iat[j,i+2] = s.item(i)

In [86]:
tap.head()


,Path,Labels,0,1,2,3,4,5,6,7,...,157,158,159,160,161,162,163,164,165,166
0,../../../Source/Clean_train_clips/Test_pad/Shu...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,../../../Source/Clean_train_clips/Test_pad/Shu...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,../../../Source/Clean_train_clips/Test_pad/Shu...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,../../../Source/Clean_train_clips/Test_pad/Shu...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,../../../Source/Clean_train_clips/Test_pad/Shu...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Add features: Short-time Fourier Transform**

In [ ]:
# Create an empty dataframe to fill with STFT values
d = pd.DataFrame(np.zeros(len(tap), (int(n_fft/2+1))))
tap = pd.concat([tap, d], axis=1)

In [ ]:
for j in range (len(tap))  :  
    s = get_features_stft(tap.loc[j,'Path'])[0]
    for i in range (int(n_fft/2+1)):
        tap.iat[j,i+2] = s.item(i)

In [ ]:
tap.head()

### Export Feature DataFrame

In [112]:
tap = tap.sample(frac=1)
tap.head()

,Path,Labels,0,1,2,3,4,5,6,7,...,10,11,12,13,14,15,16,17,18,19
88,../../../Source/Clean_train_clips/Test_pad/Bal...,0,-293.037943,58.505334,-48.589866,17.169116,-6.572513,7.458624,-18.516129,-4.076180,...,-11.396065,-6.967381,-13.589901,-5.837967,-5.179842,-7.704733,-7.414687,-3.320110,-5.325567,-0.951670
46,../../../Source/Clean_train_clips/Test_pad/Shu...,1,-384.336383,47.068545,-45.293521,13.427367,-13.270577,-1.291721,-13.609023,3.785221,...,-10.857611,-4.648366,-7.143908,-5.901735,-5.324317,-5.204102,-2.262903,-4.743410,-4.527111,-4.428521
85,../../../Source/Clean_train_clips/Test_pad/Bal...,0,-520.247082,27.081231,-20.913136,6.208832,-1.711949,3.825482,-8.540246,-3.395663,...,-5.470911,-2.400662,-2.123239,-4.389208,-4.464401,-2.490282,-1.779873,-3.779712,-1.225601,-1.614159
59,../../../Source/Clean_train_clips/Test_pad/Shu...,1,-308.469831,56.266528,-82.179828,26.003649,-6.314797,1.156485,-14.727843,-3.775998,...,-18.237127,-3.376053,-11.601492,-3.089659,-6.187914,-4.790451,-3.271651,-3.614034,-4.755073,-3.978672
57,../../../Source/Clean_train_clips/Test_pad/Shu...,1,-331.345416,62.737518,-70.032324,17.782250,-3.121940,-4.137454,-15.838885,0.413734,...,-16.465318,-1.578397,-10.076773,-4.570083,-4.776446,-6.193735,-4.736327,-5.317162,-4.541855,-3.242185


In [113]:
#tap.to_csv('../../../Source/Data/TEST_data_pad_mfcc.csv', index=None)

### Split into inputs and labels (and train and validate)

In [114]:
X = tap.drop(['Labels'], axis =1)
y = tap[['Labels']]

In [ ]:
#X_train, X_val, y_train, y_val = train_test_split(X, y,
#                                                    stratify=y, 
#                                                    test_size=0.25)

In [115]:
X_train = pd.DataFrame(X)
y_train = pd.DataFrame(y)

#X_val = pd.DataFrame(X_val)
#y_val = pd.DataFrame(y_val)



In [116]:
# Export training data
#X_train.to_csv('../../../Source/Data/X_train_audio_reaugmented_pad_mfcc.csv', index=None)
#y_train.to_csv('../../../Source/Data/y_train_audio_reaugmented_pad_mfcc.csv', index=None)

# Export validation data
#X_val.to_csv('../../../Source/Data/X_val_audio_augmented_.csv', index=None)
#y_val.to_csv('../../../Source/Data/y_val_audio_augmented_.csv', index=None)

# Export test data
#X_train.to_csv('../../../Source/Data/X_test_audio_pad_mfcc.csv', index=None)
#y_train.to_csv('../../../Source/Data/y_test_audio_pad_mfcc.csv', index=None)

## TBD "Features"

** Add features: root mean square energy value**

In [ ]:
def get_features_rmse(path):
    samples, sample_rate = librosa.load(path)
    #return np.mean(librosa.feature.rmse(y=samples).T,axis=0).item()
    return librosa.feature.rmse(samples, frame_length=512, hop_length=256)

#tap['RMSE'] = [get_features_rmse(tap.loc[idx, 'Path']) for idx in range (len(tap))]
get_features_rmse('../../../Source/Shuffle_test/1/1_pad.wav').shape

** Add features: short term energy**

In [ ]:
def get_features_ste(path):
    hop_length = 125
    frame_length = 250
    samples, sample_rate = librosa.load(path)
    energy = np.array([sum(abs(samples[i:i+frame_length]**2))for i in range(0, len(samples), hop_length)])
    #return np.mean(energy.T)
    return energy

#tap['STE'] = [get_features_ste(tap.loc[idx, 'Path']) for idx in range (len(tap))]
get_features_ste('../../../Source/Shuffle_test/1/1_pad.wav').shape

**Add features: Fast Fourier Transform**

In [ ]:
def get_features_fft(path):
    y, sr = librosa.load(path)
    return np.mean(fft(y).real)

#tap['FFT'] = [get_features_fft(tap.loc[idx, 'Path']) for idx in range (len(tap))]

**Add features: Discrete cosine transform**

In [ ]:
def get_features_dct(path):
    y, sr = librosa.load(path)
    return (dct(y))

#tap['DCT'] = [get_features_dct(tap.loc[idx, 'Path']) for idx in range (len(tap))]

**Other (maybe useful) stuff from librosa**

In [ ]:
librosa.core.get_duration(y=y, sr=sr) # Get duration in seconds

** Other code to consider/incorporate ** 

In [ ]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=20).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz

## Trying out stuff

In [ ]:
# Can I make a better graph by splitting differently? 
# Tested out in Data_Collection_Audio, ultimately ending with detect nonsilence method, then padding
samples, sample_rate = librosa.load('../../../Source/Shuffle_test/1/1.wav')
librosa.display.waveplot(samples, sr=sample_rate);



In [ ]:
# Pad with silence
padded_signal = librosa.util.pad_center(samples, 22050, axis=0) # Make clip 1 sec long
sound_clip = padded_signal
librosa.display.waveplot(sound_clip, sr=sample_rate);
#librosa.output.write_wav('../../../Source/Shuffle_test/1/1_pad.wav', padded_signal, 22050)

In [ ]:
def windows(data, window_size):
    start=0
    while start < len(data):
        yield start, start + window_size
        start += (window_size/2)

In [ ]:
i = 0
for (start, end) in windows(sound_clip, window_size):
    i +=1
print(i)

In [ ]:
log_specgrams = []
window_size = 100 * 60
sound_clip = padded_signal
bands = 41
frames = 60

In [ ]:
for (start, end) in windows(sound_clip, window_size):
    if len(sound_clip[int(start):int(end)]) == (window_size):
        signal = sound_clip[int(start):int(end)]
        melspec = librosa.feature.melspectrogram(signal,n_mels=bands)
        logspec = librosa.amplitude_to_db(melspec)
        logspec = logspec.T.flatten()[:, np.newaxis].T
        log_specgrams.append(logspec)
        
log_specgrams = np.array(log_specgrams)
print(log_specgrams.shape) #  This is (# of complete windows, 1, # of bands)
#log_specgrams = np.asarray(log_specgrams).reshape(log_specgrams.shape[0], bands, frames)

